In [17]:
import torchvision.transforms as transforms
from IPython.display import display
from PIL import Image
import os
import numpy as np
import pandas as pd
import os
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision import datasets, models
from torch.utils.data import Dataset, DataLoader
from Tool.nt_xent import nt_xent as ntX
from Tool.readfile import readfile_train as Rfile_train
from Tool.readfile import readfile_test as Rfile_test
from Tool.Lookahead import Lookahead
from Tool.data_parallel_my_v2 import BalancedDataParallel
from Tool.KNN import KNN
from model import restnet50
from Run_test import img_Dataset

In [11]:
def train(net, trainLoader1,trainLoader2, optimizer, criterion,patience,scheduler, epochs ):

    testAccuracy  = 0
    trigger_times = 0
    last_loss     = 0

    best_loss     = 100000
    for i in range(0,epochs+1):

        net.train()

        totalLoss    = 0
        accuracy     = 0
        count        = 0
        

        for idx ,data in  enumerate(zip(trainLoader1,trainLoader2)):
            
            data_1 = data[0].cuda()
            data_2 = data[1].cuda()

            optimizer.zero_grad()


            data_1 = data_1.float()
            data_2 = data_2.float()


            output_1    =   net(data_1)
            output_2    =   net(data_2)


            loss        =   criterion(output_1 , output_2)
            totalLoss   +=  loss.item()

            scheduler.step(loss.item())
            loss.backward()
            optimizer.step()
        

        print(f"Epoch [{epochs}/{i}] , total loss = {totalLoss/(len(trainLoader1.dataset)/trainLoader1.batch_size)}")


        current_loss = totalLoss/(len(trainLoader1.dataset)/trainLoader1.batch_size)
        



        if current_loss > last_loss:

            trigger_times += 1

            print('trigger times:', trigger_times)

            if trigger_times >= patience:

                print(f"Early stopping! at {i+1} :)) ")

                return bestModel

        else:
            print('reset trigger : 0')

            trigger_times =0

            
        last_loss = current_loss



        if best_loss >= current_loss:

            best_loss = current_loss
            bestModel = net

            print("Saving best model")

            torch.save(bestModel.state_dict(), "model/epoch"+str(i+1)+"_"+str(best_loss)+".pt")
            bset_model_path = "model/epoch"+str(i+1)+"_"+str(best_loss)+".pt"
        

    print("Complete training :)")


    return bestModel

In [ ]:
def valid_test(model , test_loader,y_test):

    model.eval()

    num  = torch.zeros(500 ,512)

    count =0

    with torch.no_grad():

        for test_samples,_ in test_loader:

            test_samples = test_samples.cuda()

            test_outputs = model(test_samples)

            for i in range(len(test_outputs)):

                num[count] = test_outputs[i]

                count +=1
        
    acc = KNN(num , torch.tensor(y_test), batch_size=32)

    return acc

    

In [ ]:
def fine_tune(net, test_loader,optimizer,criterion,scheduler,patience,epochs):
    
    trigger_times = 0
    last_acc      = 0
    best_acc      = 0

    for i in range(0,epochs+1):
        
        net.train()

        totalLoss    = 0
        accuracy     = 0
        count        = 0

        for idx ,(data,label) in  enumerate(test_loader):
            
            data  = data.cuda()
            label = label.cuda()
            optimizer.zero_grad()

            data_= data.float()

            output    =   net(data, label = True)
            
            loss          = criterion(output , label)

            Total_output  = torch.argmax(output, dim=1)

            accuracy   +=  (Total_output == label).sum().item()

            totalLoss  +=  loss.item()

            scheduler.step(loss.item())
            loss.backward()
            optimizer.step()

        
        valid_acc = valid_test(model = net , test_loader = test_loader , y_test = y_test)
        
        print(f"Epoch [{epochs}/{i}] , total loss = {totalLoss/(len(test_loader.dataset)/test_loader.batch_size)} acc = {accuracy/len(test_loader.dataset)} , valid acc = {valid_acc}")



        if valid_acc < last_acc:


            trigger_times += 1

            print('trigger times:', trigger_times)

            if trigger_times >= patience:

                print(f"Early stopping! at {i+1} :)) ")

                return net

        else:
            
            print("reset trigger :)) ")
            trigger_times=0
            
        last_acc = valid_acc


        if best_acc < valid_acc:

            best_acc = valid_acc
            
            bestModel = net

            print("Saving best model")

            torch.save(bestModel.state_dict(), "model/epoch"+str(i+1)+"_"+str(best_acc)+".pt")
            bset_model_path = "model/epoch"+str(i+1)+"_"+str(best_acc)+".pt"
    



    print("Complete training :)")




    return bestModel


In [16]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

criterion = ntX

lookahead = Lookahead(optimizer, k=5, alpha=0.5) 

scheduler = optim.lr_scheduler.ReduceLROnPlateau(lookahead, mode='min', factor=0.5, patience=2, min_lr=0.000001)

model_ft =  train(model,train_loader1,train_loader2,lookahead,criterion,3,scheduler,600)

Epoch [600/0] , total loss = 4.56594695919404
trigger times: 1
Saving best model
Epoch [600/1] , total loss = 4.541526796525559
reset trigger : 0
Saving best model
Epoch [600/2] , total loss = 4.5627070291747716
trigger times: 1
Epoch [600/3] , total loss = 4.552436952039186
reset trigger : 0


KeyboardInterrupt: 

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

criterion = nn.CrossEntropyLoss()

lookahead = Lookahead(optimizer, k=3, alpha=0.3) 

scheduler = optim.lr_scheduler.ReduceLROnPlateau(lookahead, mode='max', factor=0.5, patience=2, min_lr=0.000001)
model_fine_tune = fine_tune(model_ft,test_loader,lookahead,criterion,scheduler,4,200)

Epoch [200/0] , total loss = 4575.598657226563 acc = 0.252 , valid acc = 0.4359999895095825
reset trigger :)) 
Saving best model
Epoch [200/1] , total loss = 2917.576123046875 acc = 0.256 , valid acc = 0.4320000112056732
trigger times: 1
Epoch [200/2] , total loss = 2828.180421875 acc = 0.238 , valid acc = 0.414000004529953
trigger times: 2
Epoch [200/3] , total loss = 2882.7188627929686 acc = 0.252 , valid acc = 0.43799999356269836
reset trigger :)) 
Saving best model
Epoch [200/4] , total loss = 2909.447900390625 acc = 0.242 , valid acc = 0.44600000977516174
reset trigger :)) 
Saving best model
Epoch [200/5] , total loss = 2887.913466430664 acc = 0.262 , valid acc = 0.42800000309944153
trigger times: 1
Epoch [200/6] , total loss = 2917.3629497070315 acc = 0.248 , valid acc = 0.40799999237060547
trigger times: 2


KeyboardInterrupt: 

In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
def run_pred(model,path)

    predictions = []
    label = []

    print("start run predict...")

    model_ft.eval()
    prediction2 = []


    num  = torch.zeros(500 ,512)

    count =0

    with torch.no_grad():

        for test_samples,label in test_loader:
            test_samples = test_samples.cuda()

            test_outputs = model_ft(test_samples)

            for i in range(len(test_outputs)):

                num[count] = test_outputs[i]

                count +=1

    acc = KNN(num , torch.tensor(y_test), batch_size=32)

    print(acc)

    return acc

start run predict...


0.42399999499320984
